In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import psutil
from IPython.core.display import display, HTML
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))



In [2]:
tickets = pd.read_csv('data/hacknight_ticket_sample_data_2015.csv', low_memory=False, parse_dates=['issue_date', 'ticket_queue_date'])



Ok let's look for a disparity in terms of notice_level. 



Is there a certain fine_level that triggers a certain notice level? Is it the same for everyone? Maybe we could start with who is getting more boots on their cars. 


___

But first, let's take a look at the data. Next couple of lines look at the ticket sample itself from 2015 and then load in the geocode data and addresses.

In [3]:
tickets.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 23 columns):
ticket_number            50000 non-null int64
issue_date               50000 non-null datetime64[ns]
violation_location       50000 non-null object
license_plate_number     50000 non-null object
license_plate_state      49998 non-null object
license_plate_type       49383 non-null object
zipcode                  41550 non-null object
violation_code           50000 non-null object
violation_description    50000 non-null object
unit                     50000 non-null int64
unit_description         50000 non-null object
vehicle_make             50000 non-null object
fine_level1_amount       50000 non-null int64
fine_level2_amount       50000 non-null int64
current_amount_due       50000 non-null float64
total_payments           50000 non-null float64
ticket_queue             50000 non-null object
ticket_queue_date        50000 non-null datetime64[ns]
notice_level             36707 

In [4]:
tickets_geo = pd.read_csv('data/hacknight_sample_data_geocode.csv', low_memory=False)


In [5]:
tickets_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20608 entries, 0 to 20607
Data columns (total 3 columns):
address    20608 non-null object
lat        20608 non-null float64
lng        20608 non-null float64
dtypes: float64(2), object(1)
memory usage: 483.1+ KB


Ok, so I want to know what it takes to get a boot put on your car.

So I want to select all the boots in the dataset. This would be notice_level = (“SEIZ” indicates the vehicle is on the city’s boot list)

I also want to know what kind of offense triggers a boot. This is violation_description. 

And I also want to know how much someone typically owes when they get a boot. The total that they owe at the time of the violation is current_amount_due: total amount due for that citation and any related late penalties as of May 14, 2018, when data was last updated.

Ok, and we're off!

First, filter the tickets dataset to just be the ones with the boots on the car.

In [6]:
tickets[tickets.notice_level == "SEIZ" or ""]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
tickets_grouped = tickets.groupby(["violation_description", "current_amount_due"])


In [ ]:
tickets["violation_description"]["current_amount_due" > "0"]

I'm also wondering if, as a stylist told me, they tow cars without booting them first more often on the southside. Is there any truth to this claim? 

Does the city ALWAYS follow the procedure (as outlined in the PPIL article: https://features.propublica.org/driven-into-debt/chicago-ticket-debt-bankruptcy/) of more than 1 unpaid ticket, then a boot, then 24 hours to remove the boot and pay fines or get towed?

In other words, what does it take to get a car towed by the city, and are there any discrepancies in that betwee the north and the south sides of the city?

Here's the Chicago city code about when a vehicle can be towed: https://www.cityofchicago.org/content/dam/city/depts/streets/supp_info/ChicagoMunicipalCode992030.pdf

https://www.cityofchicago.org/city/en/depts/streets/supp_info/common_towing_questions.html

Seems pretty broad. Especially the esoteric city codes (in violation of Section....c)
"When an unattended vehicle is parked in violation of Section 9-40-060, 9-64-020; 9-64-050, 9-64-070, 9-64-100, 9-64-110, 9-64-120, 9-64-130(b), 9-64-140(b), 9-64-150(b), 9-64-160(b), 9-64-170, 9-64-210, 9-80-080(a), or 9-80-130;"


Maybe narrow it down by type of violation that matches this?





Since I don't have the towing data yet, just looking at SEIZ data.

Look at if the SEIZ always follows the city protocol: https://www.cityofchicago.org/city/en/depts/fin/supp_info/revenue/boot_tow_information/booted_vehicle_information.html

In other words:
-is it always unpaid 2 tickets in final determination status for over a year that lead to a boot?

-is it always 3 unpaid tickets in final determination status that lead to a boot?

--are there a few officers who give boots outside of city protocol?